In [47]:
import re
from collections import namedtuple

In [48]:
dig_plan = []
with open("input.txt", "rt") as f:
    for line in f.read().strip().split("\n"):
        direction, steps, color = re.match("^([UDRL]) (\d+) \((#[\dabcdef]+)\)", line).groups()
        dig_plan.append((direction, int(steps), color))

In [49]:
Point = namedtuple("Point", ["x", "y"])


def up(p: Point, n: int = 1) -> Point:
    return Point(p.x, p.y - n)


def left(p: Point, n: int = 1) -> Point:
    return Point(p.x - n, p.y)


def right(p: Point, n: int = 1) -> Point:
    return Point(p.x + n, p.y)


def down(p: Point, n: int = 1) -> Point:
    return Point(p.x, p.y + n)

# Part 1

In [50]:
position = Point(0, 0)
trenches = set()

for direction, steps, _ in dig_plan:
    for _ in range(steps):
        if direction == "U":
            position = up(position)
        elif direction == "L":
            position = left(position)
        elif direction == "R":
            position = right(position)
        elif direction == "D":
            position = down(position)
        trenches.add(position)

max_outside_x = max(p[0] for p in trenches) + 1
max_outside_y = max(p[1] for p in trenches) + 1
min_outside_x = min(p[0] for p in trenches) + 1
min_outside_y = min(p[1] for p in trenches) + 1

# Find a position inside the trench

first_inside_position = None
to_check = [Point(max_outside_x, max_outside_y)]
visited = set()
while first_inside_position is None:
    outside_position = to_check.pop()
    visited.add(outside_position)
    
    for direction in (up, left, right, down):
        one_step = direction(outside_position)
        two_step = direction(one_step)

        if one_step in trenches and two_step not in trenches:
            first_inside_position = two_step
            break

        if (
            one_step.x <= max_outside_x
            and one_step.x >= min_outside_x
            and one_step.y <= max_outside_y
            and one_step.y >= min_outside_y
            and one_step not in visited
        ):
            to_check.append(one_step)

# Flood fill

filled = {first_inside_position}
to_check = [first_inside_position]
visited = set()
while to_check:
    inside_position = to_check.pop()
    visited.add(inside_position)

    for direction in (up, left, right, down):
        one_step = direction(inside_position)

        if one_step not in visited and one_step not in trenches:
            filled.add(one_step)
            to_check.append(one_step)

len(filled) + len(trenches)

108909

# Part 2

In [51]:
new_dig_plan = []
directions_map = {
    "0": "R",
    "1": "D",
    "2": "L",
    "3": "U",
}
for *_, color in dig_plan:
    direction = directions_map[color[-1]]
    steps = int(color[1:-1], 16)
    new_dig_plan.append((direction, steps, color))


trenches = []
position = Point(0, 0)
for direction, steps, _ in new_dig_plan:
    if direction == "U":
        position = up(position, steps)
    elif direction == "L":
        position = left(position, steps)
    elif direction == "R":
        position = right(position, steps)
    elif direction == "D":
        position = down(position, steps)
    trenches.append(position)


def shoelace_formula(points: list[Point]) -> float:
    area = 0.0
    p = points[-1]
    for q in points:
        area += (p.x + q.x) * (p.y - q.y)
        p = q
    return int(abs(area / 2.0))


shoelace_area = shoelace_formula(trenches)
borders_left = sum(steps for direction, steps, _ in new_dig_plan if direction in "DL")

shoelace_area + borders_left + 1

133125706867777